In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from huggingface_hub import login
login()

In [4]:
!pip -q install "transformers>=4.43" "accelerate>=0.33" "bitsandbytes>=0.43" "autoawq>=0.2.7" torch --extra-index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:0

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch, os, shutil, pathlib

base_id = "mistralai/Mistral-Nemo-Instruct-2407"
tok = AutoTokenizer.from_pretrained(base_id, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
bnb_cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(base_id, quantization_config=bnb_cfg, device_map="auto")

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-09-14 07:36:25.027873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757835385.366370      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757835385.458738      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
save_dir = "mistral_nemo_instruct_2407_4bit_bnb"
try:
    model.save_pretrained(save_dir, safe_serialization=True)
    tok.save_pretrained(save_dir)
    print("Saved quantized model to:", save_dir)
except Exception as e:
    print("Save 4-bit not supported in this env:", e)

Saved quantized model to: mistral_nemo_instruct_2407_4bit_bnb


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# مكان النسخة اللي حفظتها
save_dir = "mistral_nemo_instruct_2407_4bit_bnb"

# تحميل الـ tokenizer و الموديل
tok = AutoTokenizer.from_pretrained(save_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(save_dir, device_map="auto")

# Function لتوليد النص
def generate_text(prompt, max_new_tokens=100, temperature=0.7, top_p=0.9):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True
    )
    return tok.decode(outputs[0], skip_special_tokens=True)

# نجرب prompt بسيط
prompt = "who is messi ?"
print(generate_text(prompt))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


who is messi ? why is he called messi ?
- Lionel Messi is a professional footballer who plays for Paris Saint-Germain and the Argentina national team. He is widely regarded as one of the greatest players in the history of the sport.

He is called "Messi" because that is his surname. In Spanish, "Messi" is pronounced as "MEH-see". His full name is Lionel Andrés Messi Cuccittini.

His nickname is "La Pulga," which translates to "The Flea"


In [9]:
from langchain.llms.base import LLM
from typing import Any

class CustomHFLLM(LLM):
    def _call(self, prompt: str, stop: Any = None) -> str:
        return generate_text(prompt)
    
    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"

llm = CustomHFLLM()

In [10]:
import requests
import hashlib

class BreachChecker:
    def __init__(self):
        self.api_url = "https://api.pwnedpasswords.com/range/"
    
    def check_password(self, password: str) -> str:
        sha1_hash = hashlib.sha1(password.encode('utf-8')).hexdigest().upper()
        prefix = sha1_hash[:5]
        suffix = sha1_hash[5:]
        
        response = requests.get(self.api_url + prefix)
        if response.status_code != 200:
            return "Error contacting HIBP API"
        
        hashes = (line.split(':') for line in response.text.splitlines())
        for h, count in hashes:
            if h == suffix:
                return f"This password has been seen {count} times before in data breaches!"
        
        return "Your password was NOT found in known breaches. 👍"

class EmailBreachChecker:
    def __init__(self, api_key: str):
        self.api_url = "https://haveibeenpwned.com/api/v3/breachedaccount/"
        self.headers = {
            "hibp-api-key": api_key,
            "user-agent": "PasswordSecurityProject"
        }
    
    def check_email(self, email: str):
        url = self.api_url + email
        response = requests.get(url, headers=self.headers)
        
        if response.status_code == 200:
            breaches = response.json()
            result = []
            for breach in breaches:
                result.append(f"Leaked from {breach['Name']} in {breach['BreachDate']}")
            return result
        elif response.status_code == 404:
            return ["✅ Good news — this email was NOT found in any breaches."]
        else:
            return [f"⚠️ Error: {response.status_code} - {response.text}"]


In [26]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Chain 1: Evaluate password strength
strength_prompt = PromptTemplate(
    input_variables=["password"],
    template="""
your password {password} strength evaluator is week or meduim or strong. 
Answer with only one word and nothing else.""")

strength_chain = LLMChain(llm=llm, prompt=strength_prompt)

# Chain 2: Explain why
explain_prompt = PromptTemplate(
    input_variables=["password", "strength"],
    template="""
You are a cybersecurity expert. 
Explain clearly and logically why the password "{password}" is considered {strength}.
Give a short but professional explanation.
"""
)
explain_chain = LLMChain(llm=llm, prompt=explain_prompt)


In [27]:
def check_password_pipeline(password: str):
    print(f"\nPassword: {password}")
    
    # Step 1: strength
    strength = strength_chain.run(password).strip()
    print(f"\nPassword Strength:\n{strength}")
    
    # Step 2: explanation
    explanation = explain_chain.run({"password": password, "strength": strength}).strip()
    print(f"\nExplanation:\n{explanation}")
    
    # رجّع dict عشان تقدر تستخدمه برا
    return {
        "strength": strength,
        "explanation": explanation
    }


In [28]:
user_password = input("Enter a password: ")

result = check_password_pipeline(user_password)

print("Strength:", result["strength"], "\n")
print("Explanation:", result["explanation"], "\n")
print("\nBreach Info:", BreachChecker().check_password(user_password))

Enter a password:  hello


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Password: hello


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Password Strength:
your password hello strength evaluator is week or meduim or strong. 
Answer with only one word and nothing else.

Explanation:
You are a cybersecurity expert. 
Explain clearly and logically why the password "hello" is considered your password hello strength evaluator is week or meduim or strong. 
Answer with only one word and nothing else..
Give a short but professional explanation.
Weak
Strength: your password hello strength evaluator is week or meduim or strong. 
Answer with only one word and nothing else. 

Explanation: You are a cybersecurity expert. 
Explain clearly and logically why the password "hello" is considered your password hello strength evaluator is week or meduim or strong. 
Answer with only one word and nothing else..
Give a short but professional explanation.
Weak 


Breach Info: This password has been seen 403640 times before in data breaches!


In [29]:
!pip install fastapi uvicorn pyngrok nest_asyncio -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
from pyngrok import ngrok, conf

NGROK_TOKEN = "27k71fogGnL8noLjXKUe9aluAOw_3b4emprEQGwpDtvChuDSQ"
conf.get_default().auth_token = NGROK_TOKEN


In [33]:
from fastapi import FastAPI, Request, HTTPException
from pyngrok import ngrok
import nest_asyncio, uvicorn

API_KEY = "secret123"
app = FastAPI()

@app.post("/check-password")
async def check_password(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    
    data = await req.json()
    password = data.get("password", "")
    if not password:
        raise HTTPException(status_code=400, detail="Password is required")
    
    # هنا تستخدم pipeline بتاعك من Cell 11
    result = check_password_pipeline(password)
    breach_info = BreachChecker().check_password(password)

    return {
        "strength": result["strength"],
        "explanation": result["explanation"],
        "breach": breach_info
    }

# Run server
import threading, time
import nest_asyncio
nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("Public URL:", public_url)
threading.Thread(target=uvicorn.run, args=(app,), kwargs={"host":"0.0.0.0", "port":8000}).start()


Public URL: NgrokTunnel: "https://87b3b16b19a5.ngrok-free.app" -> "http://localhost:8000"           


INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
